In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

In [2]:
from gensim.models.keyedvectors import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format(r'D:\迅雷下载\GoogleNews-vectors-negative300.bin.gz',binary=True,limit=200000)
word_vectors.most_similar(positive=['cooking','potatoes'],topn=5)     #cooking和potatoes两个词向量相加
word_vectors.most_similar(negative=['cooking','potatoes'],topn=2)     #cooking和potatoes两个词向量相减
word_vectors.doesnt_match('potatoes milk cake computer'.split())      #检测不相关的词向量
word_vectors.most_similar(positive=['king','woman'],negative=['man'],topn=2)   #king+woman-man
word_vectors.similarity('princess','queen')     #计算两个词项之间的相似度
# word_vectors['phone']        #查看phone的向量

[('cook', 0.6973531246185303),
 ('sweet_potatoes', 0.6600280404090881),
 ('vegetables', 0.6513738632202148),
 ('onions', 0.6512383818626404),
 ('baking', 0.6481685042381287)]

[('AP_TECHNOLOGY_NEWS', 0.15106438100337982), ('ATLAS', 0.14635424315929413)]

C:\Users\24132\AppData\Roaming\Python\Python38\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'computer'

[('queen', 0.7118192315101624), ('monarch', 0.6189672946929932)]

0.70705307

# 训练word2vec模型

In [46]:
import multiprocessing as mul
from gensim.models.word2vec import Word2Vec

#将文章分隔成转换为词向量需要的数据格式
file = r'C:\Users\24132\Desktop\1.txt'
with open(file,'r',encoding='utf-8')as f:
    a = ''.join(f.read().split('\n'))
    b = ''.join(a.split('\u3000\u3000')).split('.')
token_list = []
from nltk.tokenize import TreebankWordTokenizer    #内置多种英语常见的分词规则
tokenizer = TreebankWordTokenizer()
for i in b:
    token_list.append(tokenizer.tokenize(i))

#训练Word2vec模型
num_features = 300
min_word_count = 1
num_workers = mul.cpu_count()   #动态设置cpu核数
window_size = 6
subsampling = 1e-3

model = Word2Vec(
    token_list,
    workers = num_workers,
    size = num_features,
    min_count = min_word_count,
    window = window_size,
    sample = subsampling
)

# model.init_sims(replace=True)    #模型训练完毕时使用，存储隐藏层的权重并丢弃用于预测共现词的输出权重

model_name = r'D:\深度学习模型\word2vec_model_mine_1'
model.save(model_name)
    

In [47]:
from gensim.models.word2vec import Word2Vec
model_name = r'D:\深度学习模型\word2vec_model_mine_1'
model = Word2Vec.load(model_name)
model.most_similar('forever')

<ipython-input-47-5ddf45db7c4d>:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('forever')


[('of', 0.17407584190368652),
 ('an', 0.17077642679214478),
 ('neither', 0.1651541292667389),
 ('never', 0.16309814155101776),
 ('Then', 0.1622140109539032),
 ('discipline', 0.14105772972106934),
 ('arise', 0.1328985095024109),
 ('act', 0.12999625504016876),
 ('oath', 0.12474499642848969),
 ('difference', 0.12166089564561844)]

# 词关系可视化

In [1]:
import os
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
wv = KeyedVectors.load_word2vec_format(r'D:\迅雷下载\GoogleNews-vectors-negative300.bin.gz',binary=True,limit=300000)
len(wv.vocab)
vocab = pd.Series(wv.vocab)
vocab.iloc[100000:100006]

distinctiveness    Vocab(count:200000, index:100000)
Namco_Bandai       Vocab(count:199999, index:100001)
ramparts           Vocab(count:199998, index:100002)
Linden_Lab         Vocab(count:199997, index:100003)
Revolutions        Vocab(count:199996, index:100004)
Henderson_Nev.     Vocab(count:199995, index:100005)
dtype: object

In [6]:
import numpy as np
len(wv['ramparts']),type(wv['ramparts'])    #词ramparts的向量
np.linalg.norm(wv['ramparts']-wv['Revolutions'])    #向量模长——欧基米德距离

cos_similarity = np.dot(wv['ramparts'],wv['Revolutions'])/np.linalg.norm(wv['ramparts'])/np.linalg.norm(wv['Revolutions'])
1-cos_similarity      #余弦距离

(300, numpy.ndarray)

5.167152

0.9201079830527306

In [14]:
#美国城市的数据
from nlpia.data.loaders import get_data
cities = get_data('cities_us')


INFO:nlpia.futil:Reading CSV with `read_csv(*('C:\\Users\\24132\\AppData\\Roaming\\Python\\Python38\\site-packages\\nlpia\\data\\cities_us.csv.gz',), **{'nrows': None, 'low_memory': False})`...


# 利用Doc2vec计算文档相似度

In [36]:
import multiprocessing
from gensim.models.doc2vec import TaggedDocument,Doc2Vec
from gensim.utils import simple_preprocess

num_cores = multiprocessing.cpu_count()
file = r'C:\Users\24132\Desktop\1.txt'
# with open(file,'r',encoding='utf-8')as f:
#     a = ''.join(f.read().split('\n'))
#     b = ''.join(a.split('\u3000\u3000')).split('.')
with open(file,'r',encoding='utf-8')as f:
    a = f.read()
with open(file,'r',encoding='utf-8')as f:
    b = f.read()
with open(file,'r',encoding='utf-8')as f:
    c = f.read()
    
corpus = [a,b]

train_corpus = []

for i,text in enumerate(corpus):
    tagged_doc = TaggedDocument(simple_preprocess(text),[i])
    train_corpus.append(tagged_doc)

model = Doc2Vec(size=100,min_count=2,workers=num_cores,iter=10)    #滑动窗口大小为10
# train_corpus
model.build_vocab(train_corpus)
model.train(train_corpus,total_examples=model.corpus_count,epochs=model.iter)


#将从未见过的文档来更新向量
vec_3 = model.infer_vector(simple_preprocess(c),steps=10)  #迭代10步来更新


C:\Users\24132\AppData\Roaming\Python\Python38\site-packages\gensim\models\doc2vec.py:315: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
C:\Users\24132\AppData\Roaming\Python\Python38\site-packages\gensim\models\doc2vec.py:319: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
INFO:gensim.models.doc2vec:collecting all words and their counts
INFO:gensim.models.doc2vec:PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO:gensim.models.doc2vec:collected 488 word types and 2 unique tags from a corpus of 2 examples and 2696 words
INFO:gensim.models.word2vec:Loading a fresh vocabulary
INFO:gensim.models.word2vec:effective_min_count=2 retains 488 unique words (1

INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 8 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 7 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 6 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 5 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 4 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 3 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 6 : training on 2696 raw words (1806 effective words) took 0.0s, 203470 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awai

In [46]:
import numpy as np
def l2(a,b):
    return np.linalg.norm(a-b)
def cos(a,b):
    return 1-a.dot(b)/np.linalg.norm(a)/np.linalg.norm(b)
'下面分别事原本两篇文章的距离和原文章分别与新增文章的距离：'
l2(model[0],model[1])
l2(model[0],vec_3)
l2(model[1],vec_3)


'下面分别事原本两篇文章的距离和原文章分别与新增文章的距离：'

0.5909461

2.8172793

2.2278397